In [10]:
import os
print(os.getcwd())

os.chdir(r"C:\Users\adler\OneDrive\Talent.AI\TalentAI-research-yasmin")

# Define output directory
output_dir = "."

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

C:\Users\adler\OneDrive\Talent.AI\TalentAI-research-yasmin


In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from matplotlib.colors import LinearSegmentedColormap

def process_excel_data(file_path):
    """
    Process an Excel file containing the recommendation algorithm data
    
    Parameters:
    file_path (str): Path to the Excel file
    
    Returns:
    pd.DataFrame: Processed data ready for analysis
    """
    print(f"Processing Excel file: {file_path}")
    
    # Read the Excel file
    try:
        raw_data = pd.read_excel(file_path)
        print(f"Successfully read Excel with {len(raw_data)} rows")
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        print("Trying to read as TSV file...")
        try:
            raw_data = pd.read_csv(file_path, sep='\t')
            print(f"Successfully read TSV with {len(raw_data)} rows")
        except Exception as e2:
            print(f"Error reading TSV file: {e2}")
            return pd.DataFrame()
    
    # List of all companies that appear in the data
    companies = ['Adobe', 'Amazon', 'Apple', 'Facebook', 'Google', 'IBM', 
                'Microsoft', 'Nvidia', 'Oracle', 'Salesforce', 'Tesla', 'Twitter', 'Uber']
    
    # Initialize lists to store processed data
    all_data_points = []
    
    # Only process rows up to 184 (your actual data rows)
    max_row = min(184, len(raw_data))
    
    # Process each row
    for idx in range(max_row):
        row = raw_data.iloc[idx]
        try:
            # Extract query information - handling it as a string
            query_str = str(row.iloc[0])  # Assuming query is in the first column
            query_match = re.search(r'\[(.*?)\]', query_str)
            if not query_match:
                # Try alternative: assume the first column is the query
                query_info = query_str
            else:
                query_info = query_match.group(1)
            
            # Extract dataset and distance function information
            dataset_variation = row.iloc[1] if len(row) > 1 else ""  # Second column
            distance_function = row.iloc[2] if len(row) > 2 else ""  # Third column
            
            # Process Multiclustering data
            for i, company in enumerate(companies):
                # Calculate column indices for rank and records
                rank_idx = 5 + (i * 2)        # Offset by 5 columns, then each company has 2 columns
                records_idx = 6 + (i * 2)
                
                # Safety check
                if rank_idx >= len(row) or records_idx >= len(row):
                    continue
                
                # Get rank and records values
                rank_val = row.iloc[rank_idx]
                records_val = row.iloc[records_idx]
                
                # Skip if None or NaN
                if pd.isna(rank_val) or pd.isna(records_val) or rank_val == 'None' or records_val == 'None':
                    continue
                    
                # Parse values
                try:
                    if isinstance(rank_val, str) and ',' in rank_val:
                        rank = int(rank_val.split(',')[0])
                    else:
                        rank = int(rank_val)
                        
                    if isinstance(records_val, str) and ',' in records_val:
                        records = int(records_val.split(',')[0])
                    else:
                        records = int(records_val)
                except (ValueError, TypeError):
                    continue
                    
                all_data_points.append({
                    'Query': query_info,
                    'Dataset': dataset_variation,
                    'Distance_Function': distance_function,
                    'Algorithm': 'Multiclustering',
                    'Company': company,
                    'Rank': rank,
                    'Records_in_Cluster': records
                })
            
            # Process Standard algorithm data
            # Find the 'Standard' column index
            standard_idx = -1
            for i, val in enumerate(row):
                if str(val).strip() == 'Standard':
                    standard_idx = i
                    break
            
            if standard_idx == -1:
                continue
                
            for i, company in enumerate(companies):
                # Calculate column indices for rank and records
                rank_idx = standard_idx + 2 + (i * 2)  # Standard + Nearest Cluster + (company index * 2 columns)
                records_idx = standard_idx + 3 + (i * 2)
                
                # Safety check
                if rank_idx >= len(row) or records_idx >= len(row):
                    continue
                
                # Get rank and records values
                rank_val = row.iloc[rank_idx]
                records_val = row.iloc[records_idx]
                
                # Skip if None or NaN
                if pd.isna(rank_val) or pd.isna(records_val) or rank_val == 'None' or records_val == 'None':
                    continue
                    
                # Parse values
                try:
                    if isinstance(rank_val, str) and ',' in rank_val:
                        rank = int(rank_val.split(',')[0])
                    else:
                        rank = int(rank_val)
                        
                    if isinstance(records_val, str) and ',' in records_val:
                        records = int(records_val.split(',')[0])
                    else:
                        records = int(records_val)
                except (ValueError, TypeError):
                    continue
                    
                all_data_points.append({
                    'Query': query_info,
                    'Dataset': dataset_variation,
                    'Distance_Function': distance_function,
                    'Algorithm': 'Standard',
                    'Company': company,
                    'Rank': rank,
                    'Records_in_Cluster': records
                })
                
        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            continue
    
    result_df = pd.DataFrame(all_data_points)
    print(f"Successfully created DataFrame with {len(result_df)} data points")
    return result_df

def visualize_records_vs_rank(df):
    """
    Create plots showing the relationship between Records and Rank for each algorithm
    
    Parameters:
    df (pd.DataFrame): Processed data
    """
    print("Generating records vs rank visualization...")
    
    # Create separate DataFrames for each algorithm
    multi_df = df[df['Algorithm'] == 'Multiclustering']
    std_df = df[df['Algorithm'] == 'Standard']
    
    # Calculate correlations
    multi_corr = multi_df['Records_in_Cluster'].corr(multi_df['Rank'])
    std_corr = std_df['Records_in_Cluster'].corr(std_df['Rank'])
    
    # Create a scatter plot figure
    plt.figure(figsize=(16, 8))
    
    # Plot for Multiclustering
    plt.subplot(1, 2, 1)
    plt.scatter(multi_df['Records_in_Cluster'], multi_df['Rank'], 
               alpha=0.7, s=60, c='#1f77b4')
    
    # Add regression line
    x = multi_df['Records_in_Cluster']
    y = multi_df['Rank']
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    plt.plot(np.sort(x), p(np.sort(x)), "r--", linewidth=2)
    
    # Add annotation with equation
    slope, intercept = z
    plt.text(0.05, 0.95, f"y = {slope:.3f}x + {intercept:.3f}\nr = {multi_corr:.3f}", 
             transform=plt.gca().transAxes, fontsize=12, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.title(f'Multiclustering Algorithm', fontsize=16)
    plt.xlabel('Number of Records in Cluster', fontsize=14)
    plt.ylabel('Rank (Lower is Better)', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.6)
    
    # Plot for Standard algorithm
    plt.subplot(1, 2, 2)
    plt.scatter(std_df['Records_in_Cluster'], std_df['Rank'], 
               alpha=0.7, s=60, c='#ff7f0e')
    
    # Add regression line
    x = std_df['Records_in_Cluster']
    y = std_df['Rank']
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    plt.plot(np.sort(x), p(np.sort(x)), "r--", linewidth=2)
    
    # Add annotation with equation
    slope, intercept = z
    plt.text(0.05, 0.95, f"y = {slope:.3f}x + {intercept:.3f}\nr = {std_corr:.3f}", 
             transform=plt.gca().transAxes, fontsize=12, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.title(f'Standard Algorithm', fontsize=16)
    plt.xlabel('Number of Records in Cluster', fontsize=14)
    plt.ylabel('Rank (Lower is Better)', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.6)
    
    plt.suptitle('Effect of Cluster Size on Ranking by Algorithm', fontsize=20)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'records_vs_rank_comparison.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    # Create hexbin plots for data density visualization
    plt.figure(figsize=(16, 6))
    
    # Multiclustering hexbin plot
    plt.subplot(1, 2, 1)
    hb1 = plt.hexbin(multi_df['Records_in_Cluster'], multi_df['Rank'], 
                    gridsize=15, cmap='Blues', mincnt=1)
    
    # Add regression line
    x = multi_df['Records_in_Cluster']
    y = multi_df['Rank']
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    plt.plot(np.sort(x), p(np.sort(x)), "r--", linewidth=2)
    
    plt.title(f'Multiclustering Algorithm (r={multi_corr:.3f})', fontsize=16)
    plt.xlabel('Records in Cluster', fontsize=14)
    plt.ylabel('Rank (Lower is Better)', fontsize=14)
    plt.colorbar(hb1, label='Count')
    plt.grid(alpha=0.3)
    
    # Standard algorithm hexbin plot
    plt.subplot(1, 2, 2)
    hb2 = plt.hexbin(std_df['Records_in_Cluster'], std_df['Rank'], 
                    gridsize=15, cmap='Oranges', mincnt=1)
    
    # Add regression line
    x = std_df['Records_in_Cluster']
    y = std_df['Rank']
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    plt.plot(np.sort(x), p(np.sort(x)), "r--", linewidth=2)
    
    plt.title(f'Standard Algorithm (r={std_corr:.3f})', fontsize=16)
    plt.xlabel('Records in Cluster', fontsize=14)
    plt.ylabel('Rank (Lower is Better)', fontsize=14)
    plt.colorbar(hb2, label='Count')
    plt.grid(alpha=0.3)
    
    plt.suptitle('Density of Records vs Rank by Algorithm', fontsize=20)
    plt.tight_layout()
    plt.savefig('density_plot_comparison.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print("Records vs rank visualizations saved")

def visualize_rank_distribution(df):
    """
    Create a visualization of rank distributions by algorithm
    
    Parameters:
    df (pd.DataFrame): Processed data
    """
    print("Generating rank distribution visualization...")
    
    plt.figure(figsize=(10, 6))
    
    # Create violin plots for better distribution visualization
    sns.violinplot(data=df, x='Algorithm', y='Rank', hue='Algorithm', 
                  palette=['#1f77b4', '#ff7f0e'], inner='box', legend=False)
    
    # Add boxplot inside
    sns.boxplot(data=df, x='Algorithm', y='Rank', width=0.3, color='white', 
               boxprops=dict(alpha=.5), showfliers=False)
    
    # Add strip plot for individual data points
    sns.stripplot(data=df, x='Algorithm', y='Rank', size=3, color='black', alpha=0.3, jitter=True)
    
    # Calculate and display stats
    stats = df.groupby('Algorithm')['Rank'].agg(['median', 'mean', 'count'])
    
    # Add text annotations for statistics
    y_max = df['Rank'].max()
    for i, alg in enumerate(['Multiclustering', 'Standard']):
        if alg in stats.index:
            plt.text(i, y_max * 0.9, 
                    f"Median: {stats.loc[alg, 'median']:.1f}\nMean: {stats.loc[alg, 'mean']:.1f}\nN: {stats.loc[alg, 'count']}", 
                    ha='center', va='top', fontsize=11,
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.title('Rank Distribution by Algorithm', fontsize=16)
    plt.xlabel('Algorithm', fontsize=14)
    plt.ylabel('Rank (Lower is Better)', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.4, axis='y')
    
    plt.tight_layout()
    plt.savefig('rank_distribution.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print("Rank distribution visualization saved")

def create_normalized_rank_plot(df):
    """
    Create a plot with normalized ranks to better show the bias effect
    
    Parameters:
    df (pd.DataFrame): Processed data
    """
    print("Generating normalized rank visualization...")
    
    # Create a copy of the dataframe
    df_norm = df.copy()
    
    # Calculate normalized rank
    # Formula: normalized_rank = rank / (1 + log(1 + records))
    # Lower is better - less influenced by the number of records
    df_norm['Normalized_Rank'] = df_norm['Rank'] / (1 + np.log1p(df_norm['Records_in_Cluster']))
    
    plt.figure(figsize=(10, 6))
    
    # Create violin plots for better distribution visualization
    sns.violinplot(data=df_norm, x='Algorithm', y='Normalized_Rank', hue='Algorithm',
                  palette=['#1f77b4', '#ff7f0e'], inner='box', legend=False)
    
    # Add boxplot inside
    sns.boxplot(data=df_norm, x='Algorithm', y='Normalized_Rank', width=0.3, color='white', 
               boxprops=dict(alpha=.5), showfliers=False)
    
    # Add strip plot for individual data points
    sns.stripplot(data=df_norm, x='Algorithm', y='Normalized_Rank', size=3, color='black', alpha=0.3, jitter=True)
    
    # Calculate and display stats
    stats = df_norm.groupby('Algorithm')['Normalized_Rank'].agg(['median', 'mean', 'count'])
    
    # Add text annotations for statistics
    y_max = df_norm['Normalized_Rank'].max()
    for i, alg in enumerate(['Multiclustering', 'Standard']):
        if alg in stats.index:
            plt.text(i, y_max * 0.9, 
                    f"Median: {stats.loc[alg, 'median']:.2f}\nMean: {stats.loc[alg, 'mean']:.2f}", 
                    ha='center', va='top', fontsize=11,
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.title('Normalized Rank by Algorithm (Adjusted for Cluster Size)', fontsize=16)
    plt.xlabel('Algorithm', fontsize=14)
    plt.ylabel('Normalized Rank (Lower is Better)', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.4, axis='y')
    
    plt.tight_layout()
    plt.savefig('normalized_rank.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print("Normalized rank visualization saved")

def compare_correlations_by_company(df):
    """
    Create a bar chart comparing correlation coefficients between records and rank
    for each company across both algorithms
    
    Parameters:
    df (pd.DataFrame): Processed data
    """
    print("Generating correlation comparison by company...")
    
    companies = df['Company'].unique()
    correlations = []
    
    for company in companies:
        company_data = df[df['Company'] == company]
        multi_data = company_data[company_data['Algorithm'] == 'Multiclustering']
        std_data = company_data[company_data['Algorithm'] == 'Standard']
        
        # Calculate correlations if enough data
        multi_corr = multi_data['Records_in_Cluster'].corr(multi_data['Rank']) if len(multi_data) > 2 else np.nan
        std_corr = std_data['Records_in_Cluster'].corr(std_data['Rank']) if len(std_data) > 2 else np.nan
        
        correlations.append({
            'Company': company,
            'Multiclustering': multi_corr,
            'Standard': std_corr
        })
    
    # Create DataFrame for plotting
    corr_df = pd.DataFrame(correlations)
    
    # Reshape for seaborn
    corr_melted = pd.melt(corr_df, id_vars=['Company'], 
                         value_vars=['Multiclustering', 'Standard'],
                         var_name='Algorithm', value_name='Correlation')
    
    # Create the plot
    plt.figure(figsize=(14, 8))
    
    # Sort by Standard algorithm correlation
    sorted_companies = corr_df.sort_values(by='Standard', ascending=False)['Company'].tolist()
    
    # Create a grouped bar chart
    g = sns.catplot(data=corr_melted, x='Company', y='Correlation', hue='Algorithm',
                   kind='bar', palette=['#1f77b4', '#ff7f0e'], height=6, aspect=2)
    
    # Adjust the plot
    g.set_xticklabels(rotation=45, ha='right')
    g.fig.suptitle('Correlation Between Records and Rank by Company', fontsize=16)
    g.set(xlabel='Company', ylabel='Correlation Coefficient')
    
    # Add a horizontal line at zero
    plt.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
    
    # Add overall correlations as text
    multi_overall = df[df['Algorithm'] == 'Multiclustering']['Records_in_Cluster'].corr(
        df[df['Algorithm'] == 'Multiclustering']['Rank'])
    std_overall = df[df['Algorithm'] == 'Standard']['Records_in_Cluster'].corr(
        df[df['Algorithm'] == 'Standard']['Rank'])
    
    plt.text(0.02, 0.95, f"Overall Correlations:\nMulticlustering: {multi_overall:.3f}\nStandard: {std_overall:.3f}", 
             transform=plt.gca().transAxes, fontsize=12, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.savefig('correlation_by_company.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print("Correlation comparison by company saved")

def calculate_statistics(df):
    """
    Calculate and print important statistics from the data
    
    Parameters:
    df (pd.DataFrame): Processed data
    
    Returns:
    dict: Summary statistics
    """
    print("Calculating statistics...")
    
    # Split by algorithm
    multi_df = df[df['Algorithm'] == 'Multiclustering']
    std_df = df[df['Algorithm'] == 'Standard']
    
    # Calculate overall statistics
    stats = {
        'overall': {
            'Multiclustering': {
                'count': len(multi_df),
                'avg_records': multi_df['Records_in_Cluster'].mean(),
                'avg_rank': multi_df['Rank'].mean(),
                'median_rank': multi_df['Rank'].median(),
                'correlation': multi_df['Records_in_Cluster'].corr(multi_df['Rank'])
            },
            'Standard': {
                'count': len(std_df),
                'avg_records': std_df['Records_in_Cluster'].mean(),
                'avg_rank': std_df['Rank'].mean(),
                'median_rank': std_df['Rank'].median(),
                'correlation': std_df['Records_in_Cluster'].corr(std_df['Rank'])
            }
        },
        'by_company': {}
    }
    
    # Calculate statistics by company
    for company in df['Company'].unique():
        company_multi = multi_df[multi_df['Company'] == company]
        company_std = std_df[std_df['Company'] == company]
        
        multi_corr = company_multi['Records_in_Cluster'].corr(company_multi['Rank']) if len(company_multi) > 2 else np.nan
        std_corr = company_std['Records_in_Cluster'].corr(company_std['Rank']) if len(company_std) > 2 else np.nan
        
        stats['by_company'][company] = {
            'Multiclustering': {
                'count': len(company_multi),
                'avg_records': company_multi['Records_in_Cluster'].mean() if len(company_multi) > 0 else np.nan,
                'avg_rank': company_multi['Rank'].mean() if len(company_multi) > 0 else np.nan,
                'correlation': multi_corr
            },
            'Standard': {
                'count': len(company_std),
                'avg_records': company_std['Records_in_Cluster'].mean() if len(company_std) > 0 else np.nan,
                'avg_rank': company_std['Rank'].mean() if len(company_std) > 0 else np.nan,
                'correlation': std_corr
            }
        }
    
    # Print overall statistics
    print("\nOverall Statistics:")
    print(f"Multiclustering: Records-Rank Correlation = {stats['overall']['Multiclustering']['correlation']:.3f}")
    print(f"Standard: Records-Rank Correlation = {stats['overall']['Standard']['correlation']:.3f}")
    print(f"Correlation Difference = {stats['overall']['Standard']['correlation'] - stats['overall']['Multiclustering']['correlation']:.3f}")
    
    # Create and save overall DataFrame
    overall_df = pd.DataFrame({
        'Algorithm': ['Multiclustering', 'Standard'],
        'Count': [stats['overall']['Multiclustering']['count'], stats['overall']['Standard']['count']],
        'Average Records': [stats['overall']['Multiclustering']['avg_records'], stats['overall']['Standard']['avg_records']],
        'Average Rank': [stats['overall']['Multiclustering']['avg_rank'], stats['overall']['Standard']['avg_rank']],
        'Median Rank': [stats['overall']['Multiclustering']['median_rank'], stats['overall']['Standard']['median_rank']],
        'Correlation (Records vs Rank)': [stats['overall']['Multiclustering']['correlation'], stats['overall']['Standard']['correlation']]
    })
    
    overall_df.to_csv('overall_statistics.csv', index=False)
    
    # Create and save company statistics DataFrame
    company_rows = []
    for company, company_stats in stats['by_company'].items():
        company_rows.append({
            'Company': company,
            'Multiclustering Count': company_stats['Multiclustering']['count'],
            'Standard Count': company_stats['Standard']['count'],
            'Multiclustering Avg Records': company_stats['Multiclustering']['avg_records'],
            'Standard Avg Records': company_stats['Standard']['avg_records'],
            'Multiclustering Avg Rank': company_stats['Multiclustering']['avg_rank'],
            'Standard Avg Rank': company_stats['Standard']['avg_rank'],
            'Multiclustering Correlation': company_stats['Multiclustering']['correlation'],
            'Standard Correlation': company_stats['Standard']['correlation'],
            'Correlation Difference': company_stats['Standard']['correlation'] - company_stats['Multiclustering']['correlation']
        })
    
    company_df = pd.DataFrame(company_rows)
    company_df.to_csv('company_statistics.csv', index=False)
    
    return stats

def create_analysis_report(df, stats):
    """
    Create a comprehensive analysis report in markdown format
    
    Parameters:
    df (pd.DataFrame): Processed data
    stats (dict): Statistics dictionary from calculate_statistics
    """
    print("Generating analysis report...")
    
    report_text = """# Recommendation Algorithms Analysis Report

## Overview

This analysis compares two recommendation algorithms (Multiclustering and Standard) to evaluate how the number of records in a cluster affects ranking performance. The hypothesis being tested is that the Multiclustering algorithm is less affected by cluster size bias than the Standard algorithm.

## Key Findings

"""
    
    multi_corr = stats['overall']['Multiclustering']['correlation']
    std_corr = stats['overall']['Standard']['correlation']
    corr_diff = std_corr - multi_corr
    
    report_text += f"- **Correlation between Records and Rank**:\n"
    report_text += f"  - Multiclustering: {multi_corr:.3f}\n"
    report_text += f"  - Standard: {std_corr:.3f}\n"
    report_text += f"  - Difference: {corr_diff:.3f}\n\n"
    
    if std_corr > multi_corr:
        report_text += "This confirms the hypothesis that the Standard algorithm shows a stronger correlation between the number of records in a cluster and the ranking, indicating it is more biased by cluster size.\n\n"
    else:
        report_text += "The data does not support the hypothesis that the Standard algorithm is more biased by cluster size than the Multiclustering algorithm.\n\n"
    
    report_text += "## Statistical Summary\n\n"
    report_text += "| Metric | Multiclustering | Standard |\n"
    report_text += "|--------|----------------|----------|\n"
    report_text += f"| Sample Size | {stats['overall']['Multiclustering']['count']} | {stats['overall']['Standard']['count']} |\n"
    report_text += f"| Average Records in Cluster | {stats['overall']['Multiclustering']['avg_records']:.2f} | {stats['overall']['Standard']['avg_records']:.2f} |\n"
    report_text += f"| Average Rank | {stats['overall']['Multiclustering']['avg_rank']:.2f} | {stats['overall']['Standard']['avg_rank']:.2f} |\n"
    report_text += f"| Median Rank | {stats['overall']['Multiclustering']['median_rank']:.1f} | {stats['overall']['Standard']['median_rank']:.1f} |\n"
    report_text += f"| Records-Rank Correlation | {stats['overall']['Multiclustering']['correlation']:.3f} | {stats['overall']['Standard']['correlation']:.3f} |\n\n"
    
    report_text += "## Visual Analysis\n\n"
    report_text += "The visualizations demonstrate the relationship between the number of records in a cluster and the ranking performance for each algorithm.\n\n"
    report_text += "### Records vs Rank Plots\n\n"
    report_text += "The scatter plots and density plots show how the number of records in a cluster affects ranking. The regression lines illustrate the trend, and the correlation coefficients quantify the strength of the relationship.\n\n"
    report_text += "### Rank Distribution\n\n"
    report_text += "The rank distribution plots show how rankings are distributed for each algorithm, with lower values indicating better performance.\n\n"
    report_text += "### Normalized Rank\n\n"
    report_text += "The normalized rank visualizations adjust the rank values to account for cluster size, providing a fairer comparison between the algorithms.\n\n"
    
    report_text += "## Company-Specific Analysis\n\n"
    report_text += "Different companies show varying levels of bias in the recommendation algorithms:\n\n"
    
    # Add company-specific insights
    companies_higher_std_corr = []
    companies_lower_std_corr = []
    
    for company, stats_dict in stats['by_company'].items():
        multi_corr = stats_dict['Multiclustering']['correlation']
        std_corr = stats_dict['Standard']['correlation']
        
        if not np.isnan(multi_corr) and not np.isnan(std_corr):
            if std_corr > multi_corr:
                companies_higher_std_corr.append(company)
            else:
                companies_lower_std_corr.append(company)
    
    report_text += f"- **Companies where Standard algorithm shows higher bias**: {', '.join(companies_higher_std_corr)}\n"
    report_text += f"- **Companies where Multiclustering shows higher bias or equal**: {', '.join(companies_lower_std_corr)}\n\n"
    
    report_text += "## Conclusion\n\n"
    
    if std_corr > multi_corr:
        report_text += f"The analysis confirms the hypothesis that the Standard algorithm is more influenced by the number of records in a cluster (correlation: {std_corr:.3f}) compared to the Multiclustering algorithm (correlation: {multi_corr:.3f}). This indicates that the Multiclustering approach successfully reduces bias related to cluster size in the recommendation system.\n\n"
    
    # Save the report
    with open('recommendation_analysis_report.md', 'w') as f:
        f.write(report_text)
    
    print("Analysis report saved as 'recommendation_analysis_report.md'")

def main():
    """
    Main function to run the analysis
    """
    print("Starting recommendation algorithm analysis...")
    
    # Process the data file - update this to your actual file path
    file_path = "final_measurements/unbiasing/for_company_size/with_gender_and_age_list_frequency_unbiasing_per_cluster.xlsx"
    # Uncomment the line below to use your specific path
    # file_path = "with_gender_and_age_list_frequency_unbiasing_per_cluster.xlsx"
    
    # Process the data
    df = process_excel_data(file_path)
    
    if len(df) == 0:
        print("No valid data was processed. Please check your data file format.")
        return
    
    print(f"Successfully processed data with {len(df)} data points")
    
    # Save the processed data to CSV for further analysis if needed
    df.to_csv('processed_recommendation_data.csv', index=False)
    print("Processed data saved to 'processed_recommendation_data.csv'")
    
    # Calculate statistics
    stats = calculate_statistics(df)
    
    # Create visualizations
    visualize_records_vs_rank(df)
    visualize_rank_distribution(df)
    create_normalized_rank_plot(df)
    compare_correlations_by_company(df)
    
    # Generate final report
    create_analysis_report(df, stats)
    
    print("\nAnalysis complete. All visualizations and reports have been saved.")

if __name__ == "__main__":
    main()

Starting recommendation algorithm analysis...
Processing Excel file: final_measurements/unbiasing/for_company_size/with_gender_and_age_list_frequency_unbiasing_per_cluster.xlsx
Successfully read Excel with 508 rows
Successfully created DataFrame with 3487 data points
Successfully processed data with 3487 data points
Processed data saved to 'processed_recommendation_data.csv'
Calculating statistics...

Overall Statistics:
Multiclustering: Records-Rank Correlation = -0.107
Standard: Records-Rank Correlation = -0.010
Correlation Difference = 0.097
Generating records vs rank visualization...
Records vs rank visualizations saved
Generating rank distribution visualization...
Rank distribution visualization saved
Generating normalized rank visualization...
Normalized rank visualization saved
Generating correlation comparison by company...
Correlation comparison by company saved
Generating analysis report...
Analysis report saved as 'recommendation_analysis_report.md'

Analysis complete. All v

<Figure size 1400x800 with 0 Axes>

In [22]:
def improved_scatter_comparison(df):
    """
    Create an improved scatter plot comparing both algorithms on the same plot
    with clearer regression lines and confidence intervals
    
    Parameters:
    df (pd.DataFrame): Processed data
    """
    # Create figure
    plt.figure(figsize=(12, 8))
    
    # Filter data for each algorithm
    multi_df = df[df['Algorithm'] == 'Multiclustering']
    std_df = df[df['Algorithm'] == 'Standard']
    
    # Calculate correlations
    multi_corr = multi_df['Records_in_Cluster'].corr(multi_df['Rank'])
    std_corr = std_df['Records_in_Cluster'].corr(std_df['Rank'])
    
    # Create scatter plots with alpha for better visibility of overlapping points
    plt.scatter(multi_df['Records_in_Cluster'], multi_df['Rank'], 
               alpha=0.6, s=80, color='#1f77b4', label=f'Multiclustering (r={multi_corr:.3f})')
    plt.scatter(std_df['Records_in_Cluster'], std_df['Rank'], 
               alpha=0.6, s=80, color='#ff7f0e', label=f'Standard (r={std_corr:.3f})')
    
    # Add regression lines with confidence intervals
    from scipy import stats
    
    # Multiclustering regression with confidence interval
    slope, intercept, r_value, p_value, std_err = stats.linregress(multi_df['Records_in_Cluster'], multi_df['Rank'])
    x_multi = np.linspace(multi_df['Records_in_Cluster'].min(), multi_df['Records_in_Cluster'].max(), 100)
    y_multi = slope * x_multi + intercept
    
    # Plot regression line
    plt.plot(x_multi, y_multi, color='#1f77b4', linestyle='-', linewidth=3)
    
    # Standard regression with confidence interval
    slope, intercept, r_value, p_value, std_err = stats.linregress(std_df['Records_in_Cluster'], std_df['Rank'])
    x_std = np.linspace(std_df['Records_in_Cluster'].min(), std_df['Records_in_Cluster'].max(), 100)
    y_std = slope * x_std + intercept
    
    # Plot regression line
    plt.plot(x_std, y_std, color='#ff7f0e', linestyle='-', linewidth=3)
    
    # Add shaded confidence interval (requires additional computation)
    
    # Add annotations
    plt.title('Effect of Cluster Size on Ranking: Algorithm Comparison', fontsize=18, fontweight='bold')
    plt.xlabel('Number of Records in Cluster', fontsize=14)
    plt.ylabel('Rank (Lower is Better)', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Add legend
    plt.legend(fontsize=12, loc='upper left')
    
    # Add correlation difference annotation
    correlation_diff = std_corr - multi_corr
    plt.annotate(f'Correlation Difference: {correlation_diff:.3f}',
                xy=(0.02, 0.95), xycoords='axes fraction',
                fontsize=12, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.savefig('improved_algorithm_comparison.png', dpi=300, bbox_inches='tight')
    plt.close()

In [23]:
def improved_scatter_comparison(df):
    """
    Create an improved scatter plot comparing both algorithms on the same plot
    with clearer regression lines and confidence intervals
    
    Parameters:
    df (pd.DataFrame): Processed data
    """
    # Create figure
    plt.figure(figsize=(12, 8))
    
    # Filter data for each algorithm
    multi_df = df[df['Algorithm'] == 'Multiclustering']
    std_df = df[df['Algorithm'] == 'Standard']
    
    # Calculate correlations
    multi_corr = multi_df['Records_in_Cluster'].corr(multi_df['Rank'])
    std_corr = std_df['Records_in_Cluster'].corr(std_df['Rank'])
    
    # Create scatter plots with alpha for better visibility of overlapping points
    plt.scatter(multi_df['Records_in_Cluster'], multi_df['Rank'], 
               alpha=0.6, s=80, color='#1f77b4', label=f'Multiclustering (r={multi_corr:.3f})')
    plt.scatter(std_df['Records_in_Cluster'], std_df['Rank'], 
               alpha=0.6, s=80, color='#ff7f0e', label=f'Standard (r={std_corr:.3f})')
    
    # Add regression lines with confidence intervals
    from scipy import stats
    
    # Multiclustering regression with confidence interval
    slope, intercept, r_value, p_value, std_err = stats.linregress(multi_df['Records_in_Cluster'], multi_df['Rank'])
    x_multi = np.linspace(multi_df['Records_in_Cluster'].min(), multi_df['Records_in_Cluster'].max(), 100)
    y_multi = slope * x_multi + intercept
    
    # Plot regression line
    plt.plot(x_multi, y_multi, color='#1f77b4', linestyle='-', linewidth=3)
    
    # Standard regression with confidence interval
    slope, intercept, r_value, p_value, std_err = stats.linregress(std_df['Records_in_Cluster'], std_df['Rank'])
    x_std = np.linspace(std_df['Records_in_Cluster'].min(), std_df['Records_in_Cluster'].max(), 100)
    y_std = slope * x_std + intercept
    
    # Plot regression line
    plt.plot(x_std, y_std, color='#ff7f0e', linestyle='-', linewidth=3)
    
    # Add shaded confidence interval (requires additional computation)
    
    # Add annotations
    plt.title('Effect of Cluster Size on Ranking: Algorithm Comparison', fontsize=18, fontweight='bold')
    plt.xlabel('Number of Records in Cluster', fontsize=14)
    plt.ylabel('Rank (Lower is Better)', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Add legend
    plt.legend(fontsize=12, loc='upper left')
    
    # Add correlation difference annotation
    correlation_diff = std_corr - multi_corr
    plt.annotate(f'Correlation Difference: {correlation_diff:.3f}',
                xy=(0.02, 0.95), xycoords='axes fraction',
                fontsize=12, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.savefig('improved_algorithm_comparison.png', dpi=300, bbox_inches='tight')
    plt.close()

In [24]:
def improved_scatter_comparison(df):
    """
    Create an improved scatter plot comparing both algorithms on the same plot
    with clearer regression lines and confidence intervals
    
    Parameters:
    df (pd.DataFrame): Processed data
    """
    # Create figure
    plt.figure(figsize=(12, 8))
    
    # Filter data for each algorithm
    multi_df = df[df['Algorithm'] == 'Multiclustering']
    std_df = df[df['Algorithm'] == 'Standard']
    
    # Calculate correlations
    multi_corr = multi_df['Records_in_Cluster'].corr(multi_df['Rank'])
    std_corr = std_df['Records_in_Cluster'].corr(std_df['Rank'])
    
    # Create scatter plots with alpha for better visibility of overlapping points
    plt.scatter(multi_df['Records_in_Cluster'], multi_df['Rank'], 
               alpha=0.6, s=80, color='#1f77b4', label=f'Multiclustering (r={multi_corr:.3f})')
    plt.scatter(std_df['Records_in_Cluster'], std_df['Rank'], 
               alpha=0.6, s=80, color='#ff7f0e', label=f'Standard (r={std_corr:.3f})')
    
    # Add regression lines with confidence intervals
    from scipy import stats
    
    # Multiclustering regression with confidence interval
    slope, intercept, r_value, p_value, std_err = stats.linregress(multi_df['Records_in_Cluster'], multi_df['Rank'])
    x_multi = np.linspace(multi_df['Records_in_Cluster'].min(), multi_df['Records_in_Cluster'].max(), 100)
    y_multi = slope * x_multi + intercept
    
    # Plot regression line
    plt.plot(x_multi, y_multi, color='#1f77b4', linestyle='-', linewidth=3)
    
    # Standard regression with confidence interval
    slope, intercept, r_value, p_value, std_err = stats.linregress(std_df['Records_in_Cluster'], std_df['Rank'])
    x_std = np.linspace(std_df['Records_in_Cluster'].min(), std_df['Records_in_Cluster'].max(), 100)
    y_std = slope * x_std + intercept
    
    # Plot regression line
    plt.plot(x_std, y_std, color='#ff7f0e', linestyle='-', linewidth=3)
    
    # Add shaded confidence interval (requires additional computation)
    
    # Add annotations
    plt.title('Effect of Cluster Size on Ranking: Algorithm Comparison', fontsize=18, fontweight='bold')
    plt.xlabel('Number of Records in Cluster', fontsize=14)
    plt.ylabel('Rank (Lower is Better)', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Add legend
    plt.legend(fontsize=12, loc='upper left')
    
    # Add correlation difference annotation
    correlation_diff = std_corr - multi_corr
    plt.annotate(f'Correlation Difference: {correlation_diff:.3f}',
                xy=(0.02, 0.95), xycoords='axes fraction',
                fontsize=12, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.savefig('improved_algorithm_comparison.png', dpi=300, bbox_inches='tight')
    plt.close()

In [28]:
def create_binned_analysis_plot(df):
    """
    Create a binned analysis chart showing average rank by record count bins
    
    Parameters:
    df (pd.DataFrame): Processed data
    """
    # Create figure
    plt.figure(figsize=(14, 8))
    
    # Filter data for each algorithm
    multi_df = df[df['Algorithm'] == 'Multiclustering']
    std_df = df[df['Algorithm'] == 'Standard']
    
    # Create bins for records
    bins = [0, 2, 4, 6, 8, 10, 12, 14, 16, np.inf]
    bin_labels = ['0-2', '3-4', '5-6', '7-8', '9-10', '11-12', '13-14', '15-16', '17+']
    
    # Function to create binned data
    def create_binned_data(algorithm_df):
        algorithm_df = algorithm_df.copy()
        algorithm_df['RecordBin'] = pd.cut(algorithm_df['Records_in_Cluster'], bins, labels=bin_labels)
        binned_data = algorithm_df.groupby('RecordBin').agg({
            'Rank': ['mean', 'median', 'count', 'std']
        }).reset_index()
        binned_data.columns = ['RecordBin', 'MeanRank', 'MedianRank', 'Count', 'StdRank']
        return binned_data
    
    # Create binned data for each algorithm
    multi_binned = create_binned_data(multi_df)
    std_binned = create_binned_data(std_df)
    
    # Set up bar positions
    bar_width = 0.35
    bin_positions = np.arange(len(bin_labels))
    
    # Plot bars with error bars
    plt.bar(bin_positions - bar_width/2, multi_binned['MeanRank'], 
           width=bar_width, color='#1f77b4', alpha=0.7, label='Multiclustering')
    plt.errorbar(bin_positions - bar_width/2, multi_binned['MeanRank'], 
                yerr=multi_binned['StdRank'] / np.sqrt(multi_binned['Count']), 
                fmt='none', color='black', capsize=5)
    
    plt.bar(bin_positions + bar_width/2, std_binned['MeanRank'], 
           width=bar_width, color='#ff7f0e', alpha=0.7, label='Standard')
    plt.errorbar(bin_positions + bar_width/2, std_binned['MeanRank'], 
                yerr=std_binned['StdRank'] / np.sqrt(std_binned['Count']), 
                fmt='none', color='black', capsize=5)
    
    # Add count annotations
    for i, count in enumerate(multi_binned['Count']):
        plt.text(i - bar_width/2, multi_binned['MeanRank'].iloc[i] + 0.5, 
                f'n={count}', ha='center', va='bottom', fontsize=9)
    
    for i, count in enumerate(std_binned['Count']):
        plt.text(i + bar_width/2, std_binned['MeanRank'].iloc[i] + 0.5, 
                f'n={count}', ha='center', va='bottom', fontsize=9)
    
    # Add trend lines
    multi_x = bin_positions
    multi_y = multi_binned['MeanRank']
    std_x = bin_positions
    std_y = std_binned['MeanRank']
    
    # Calculate trend lines
    multi_z = np.polyfit(multi_x, multi_y, 1)
    multi_p = np.poly1d(multi_z)
    
    std_z = np.polyfit(std_x, std_y, 1)
    std_p = np.poly1d(std_z)
    
    # Add trend lines
    plt.plot(bin_positions, multi_p(bin_positions), 'b--', linewidth=2)
    plt.plot(bin_positions, std_p(bin_positions), 'r--', linewidth=2)
    
    # Add trend line equations
    multi_slope, multi_intercept = multi_z
    std_slope, std_intercept = std_z
    
    plt.text(0.05, 0.05, 
            f"Multiclustering trend: y = {multi_slope:.3f}x + {multi_intercept:.3f}\n"
            f"Standard trend: y = {std_slope:.3f}x + {std_intercept:.3f}\n"
            f"Slope difference: {std_slope - multi_slope:.3f}",
            transform=plt.gca().transAxes, fontsize=12, verticalalignment='bottom',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Setup plot labels and styling
    plt.xlabel('Number of Records in Cluster', fontsize=14)
    plt.ylabel('Average Rank (Lower is Better)', fontsize=14)
    plt.title('Average Rank by Records in Cluster', fontsize=18, fontweight='bold')
    plt.xticks(bin_positions, bin_labels, rotation=45)
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.legend(fontsize=12)
    
    plt.tight_layout()
    plt.savefig('binned_analysis_plot.png', dpi=300, bbox_inches='tight')
    plt.close()

In [25]:
def create_company_heatmap(df):
    """
    Create a heatmap showing the relationship between records and rank across companies
    
    Parameters:
    df (pd.DataFrame): Processed data
    """
    # Create two separate plots for algorithms
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    
    # Filter data for each algorithm
    multi_df = df[df['Algorithm'] == 'Multiclustering']
    std_df = df[df['Algorithm'] == 'Standard']
    
    # Prepare data for heatmap - first create company vs records pivot tables
    multi_pivot = multi_df.pivot_table(
        index='Company', 
        values='Rank',
        columns=pd.cut(multi_df['Records_in_Cluster'], 
                      bins=[0, 3, 6, 9, 12, 15, 18], 
                      labels=['0-3', '4-6', '7-9', '10-12', '13-15', '16-18']),
        aggfunc='mean'
    )
    
    std_pivot = std_df.pivot_table(
        index='Company', 
        values='Rank',
        columns=pd.cut(std_df['Records_in_Cluster'], 
                      bins=[0, 3, 6, 9, 12, 15, 18], 
                      labels=['0-3', '4-6', '7-9', '10-12', '13-15', '16-18']),
        aggfunc='mean'
    )
    
    # Create custom colormap - lower numbers (better ranks) should be green
    colors = ['#1a9850', '#91cf60', '#d9ef8b', '#ffffbf', '#fee08b', '#fc8d59', '#d73027']
    cmap = LinearSegmentedColormap.from_list('GreenToRed', colors, N=100)
    
    # Plot heatmaps
    sns.heatmap(multi_pivot, ax=ax1, cmap=cmap, annot=True, fmt='.1f', 
               linewidths=0.5, vmin=1, vmax=16, cbar_kws={'label': 'Average Rank'})
    sns.heatmap(std_pivot, ax=ax2, cmap=cmap, annot=True, fmt='.1f', 
               linewidths=0.5, vmin=1, vmax=16, cbar_kws={'label': 'Average Rank'})
    
    # Titles and labels
    ax1.set_title('Multiclustering: Average Rank by Company and Records', fontsize=16)
    ax2.set_title('Standard: Average Rank by Company and Records', fontsize=16)
    
    for ax in [ax1, ax2]:
        ax.set_ylabel('Company', fontsize=14)
        ax.set_xlabel('Records in Cluster', fontsize=14)
    
    plt.suptitle('Heat Map: Company Performance by Cluster Size', fontsize=20, fontweight='bold')
    plt.tight_layout()
    
    plt.savefig('company_heatmap.png', dpi=300, bbox_inches='tight')
    plt.close()

In [26]:
def improved_analysis(df):
    """
    Perform an improved analysis with better visualizations
    
    Parameters:
    df (pd.DataFrame): Processed data
    """
    # Create all the improved visualizations
    improved_scatter_comparison(df)
    create_binned_analysis_plot(df)  
    create_company_heatmap(df)
    create_significance_plot(df)
    
    # Additional visualizations you can consider:
    # 1. Create a faceted plot by company
    # 2. Create a version that analyzes the effect by dataset variation or distance function
    # 3. Create an interactive version of these plots with plotly
    
    print("Improved analysis complete. All visualizations saved.")

In [29]:
def main():
    """
    Main function to run the analysis
    """
    print("Starting recommendation algorithm analysis...")
    
    # Process the data file - update this to your actual file path
    file_path = "final_measurements/unbiasing/for_company_size/with_gender_and_age_list_frequency_unbiasing_per_cluster.xlsx"
    
    # Process the data
    df = process_excel_data(file_path)
    
    if len(df) == 0:
        print("No valid data was processed. Please check your data file format.")
        return
    
    print(f"Successfully processed data with {len(df)} data points")
    
    # Save the processed data to CSV for further analysis if needed
    df.to_csv('processed_recommendation_data.csv', index=False)
    
    # Run original visualizations
    visualize_records_vs_rank(df)
    visualize_rank_distribution(df)
    create_normalized_rank_plot(df)
    compare_correlations_by_company(df)
    
    # Add improved visualizations
    improved_analysis(df)
    
    # Calculate statistics
    stats = calculate_statistics(df)
    
    # Generate final report
    create_analysis_report(df, stats)
    
    print("\nAnalysis complete. All visualizations and reports have been saved.")
    
if __name__ == "__main__": 
    main()
    

Starting recommendation algorithm analysis...
Processing Excel file: final_measurements/unbiasing/for_company_size/with_gender_and_age_list_frequency_unbiasing_per_cluster.xlsx
Successfully read Excel with 508 rows
Successfully created DataFrame with 3487 data points
Successfully processed data with 3487 data points
Generating records vs rank visualization...
Records vs rank visualizations saved
Generating rank distribution visualization...
Rank distribution visualization saved
Generating normalized rank visualization...
Normalized rank visualization saved
Generating correlation comparison by company...
Correlation comparison by company saved


C:\Users\adler\AppData\Local\Temp\ipykernel_26884\1278906343.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  binned_data = algorithm_df.groupby('RecordBin').agg({
C:\Users\adler\AppData\Local\Temp\ipykernel_26884\1278906343.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  binned_data = algorithm_df.groupby('RecordBin').agg({
C:\Users\adler\AppData\Local\Temp\ipykernel_26884\4063551401.py:16: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  mu

NameError: name 'create_significance_plot' is not defined

<Figure size 1400x800 with 0 Axes>